In [7]:
# imports
import duckdb
import ibis
import pandas as pd

In [23]:
# bigquery database connection
con = ibis.bigquery.connect(
    project_id="timformatie",
    dataset_id="test_data",
)
df = con.table("patients")
df

,patient,group,score
0,1,a,6.0
1,5,a,6.0
2,8,b,10.0
3,9,b,4.0
4,3,a,7.0
5,7,b,9.0
6,6,b,7.0
7,2,a,7.0
8,4,a,3.0
9,10,b,8.0


In [8]:
# pandas table in memory
df_pandas = pd.DataFrame(
    {
        "patient": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        "group": ["a", "a", "a", "a", "a", "b", "b", "b", "b", "b"],
        "score": [6.0, 7.0, 7.0, 3.0, 6.0, 7.0, 9.0, 10.0, 4.0, 8.0],
    }
)
df = ibis.memtable(df_pandas, name="df")
df

/Users/timeeftink/Documents/apps/hdc-data-analytics-handbook/.venv/lib/python3.11/site-packages/ibis/formats/pandas.py:27: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pdt.is_datetime64tz_dtype(typ):
/Users/timeeftink/Documents/apps/hdc-data-analytics-handbook/.venv/lib/python3.11/site-packages/ibis/formats/pandas.py:31: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pdt.is_categorical_dtype(typ):


InMemoryTable
  data:
    PandasDataFrameProxy:
         patient group  score
      0        1     a    6.0
      1        2     a    7.0
      2        3     a    7.0
      3        4     a    3.0
      4        5     a    6.0
      5        6     b    7.0
      6        7     b    9.0
      7        8     b   10.0
      8        9     b    4.0
      9       10     b    8.0

## Group by, summarise

In [24]:
df_grouped = df.group_by("group").aggregate(df.score.mean()).to_pandas()
df_grouped

,group,Mean(score)
0,a,5.8
1,b,7.6


In [25]:
df_grouped = df.group_by("group").aggregate(df.score.mean())
ibis.show_sql(df_grouped)

SELECT
  t0.`group`,
  avg(t0.`score`) AS `Mean_score`
FROM timformatie.test_data.patients AS t0
GROUP BY
  1


## Filter

In [15]:
df_filtered = df.filter(df.group == "a").to_pandas()
df_filtered

,patient,group,score
0,1,a,6.0
1,2,a,7.0
2,3,a,7.0
3,4,a,3.0
4,5,a,6.0


## Mutate

In [12]:
df_new_var = df.mutate(score_adjusted=df.score + 0.5).to_pandas()
df_new_var

,patient,group,score,score_adjusted
0,1,a,6.0,6.5
1,2,a,7.0,7.5
2,3,a,7.0,7.5
3,4,a,3.0,3.5
4,5,a,6.0,6.5
5,6,b,7.0,7.5
6,7,b,9.0,9.5
7,8,b,10.0,10.5
8,9,b,4.0,4.5
9,10,b,8.0,8.5


## Order

In [17]:
df_ordered = df.order_by(df.score).to_pandas()
df_ordered

,patient,group,score
0,4,a,3.0
1,9,b,4.0
2,1,a,6.0
3,5,a,6.0
4,2,a,7.0
5,3,a,7.0
6,6,b,7.0
7,10,b,8.0
8,7,b,9.0
9,8,b,10.0


## Rename

In [20]:
df_renamed = df.relabel({"patient": "person_id"}).to_pandas()
df_renamed

,person_id,group,score
0,1,a,6.0
1,2,a,7.0
2,3,a,7.0
3,4,a,3.0
4,5,a,6.0
5,6,b,7.0
6,7,b,9.0
7,8,b,10.0
8,9,b,4.0
9,10,b,8.0


## Join

In [21]:
df_group_pandas =pd.DataFrame(
    {
        "group": ["a", "b"],
        "group_name": ["alpha", "beta"]
    }
)
df_group = ibis.memtable(df_group_pandas, name="df_group")
df_joined = df.left_join(df_group, df.group == df_group.group).to_pandas()
df_joined

,patient,group,score,group_right,group_name
0,1,a,6.0,a,alpha
1,2,a,7.0,a,alpha
2,3,a,7.0,a,alpha
3,4,a,3.0,a,alpha
4,5,a,6.0,a,alpha
5,6,b,7.0,b,beta
6,7,b,9.0,b,beta
7,8,b,10.0,b,beta
8,9,b,4.0,b,beta
9,10,b,8.0,b,beta
